In [126]:
import copy 
import os 
import shutil
import itertools
from collections import Counter
import json
import pickle
import pprint
import pandas as pd
import numpy as np

In [127]:
import matplotlib.pyplot as plt 
plt.rcParams['figure.figsize'] = [6, 8]

In [128]:
print(os.listdir(os.getcwd()))

['.DS_Store', '.ipynb_checkpoints', '0226_pre-standard', '0409_adult-alldata', '0409_german-alldata', '0410_adult-nodata', '0410_german-nodata', '0413_reddata1000', '0501_reddata10000', '0501_reddata2000', '0501_reddata20000', '0501_reddata5000', 'analysis.ipynb', 'plotting.ipynb', 'processed_results']


In [129]:
res_dir = '0413_reddata1000'
extra = '1000-sample'
expdir = os.path.join(os.path.join(os.getcwd(), res_dir), 'causal_discovery')
savedir = os.path.join(os.path.join(os.getcwd(), res_dir), '{}_{}'.format(res_dir, extra))

if os.path.exists(savedir):
    shutil.rmtree(savedir)
os.mkdir(savedir)



# Utility Functions 

In [130]:
def dataset_name_from_unid(uid):
    if 'adult' in uid:
        return 'adult'
    if 'german' in uid:
        return 'germanCredit'
    
    assert True == False 

In [131]:
def get_hps_from_rawres(fname):
    '''rawres fname -> features'''
    unique_id = (fname.split('rawres_')[1]).split('.json')[0]
    alpha = unique_id.split('_')[0]
    feateng = unique_id.split('_')[1]
    dataset = unique_id.split('_')[2]
    seed = unique_id.split('_')[3]
    environment = unique_id.split('_')[4]
    return feateng, dataset, seed, environment
    
     #print('{}_{}_{}_{}'.format(feateng, dataset, seed, environment))

In [132]:
def str_2_pcp(pcpstr):
    pcpstr = (pcpstr.split('(')[1]).split(')')[0]
    pcpstr = pcpstr.replace(' ', '')
    ret = set(pcpstr.split(','))
    ret.discard('')
    return ret

In [133]:
import enum 
#Part 1
START_ALPHA = 1.0
FACTOR = 2
EPS = 1e-20
#Part 2
STEP = 1e-2
FACTOR2 = 2
EPS2 = 1e-10

class POS(enum.Enum):
   big = 1
   small = 2
   perf = 3

def alpha_tune(pVals, amin, flag=0):
    #First find a CP returning alpha 
    a0 = START_ALPHA
    bounds0 = [0, 100.0]
    cp_ret = False 
    while not cp_ret:
        pos = 0
        accepted = pVals[pVals['Final_tstat'] > a0]
        
        #Determine position of alpha 
        if len(accepted.index) == 0:
            pos = POS.big
        else: 
            accepted_sets = [str_2_pcp(a) for a in list(accepted.index)]
            causal_preds = set.intersection(*accepted_sets)
            if len(causal_preds) == 0:
                pos = POS.small 
            else:
                pos = POS.perf
                cp_ret = True
                continue
                
        #Determine what alpha to check next 
        if pos == POS.big:
            bounds0[1] = a0
            if a0/FACTOR <= bounds0[0]:
                a0 = a0 - abs((a0 - bounds0[0])/2)
            else:
                a0 = a0/FACTOR
        elif pos == POS.small:
            bounds0[0] = a0
            if a0 * FACTOR >= bounds0[1]:
                a0 = a0 + abs((a0 - bounds0[1])/2)
            else:
                a0 = a0 * FACTOR
        
        #Stability check in case no CPs 
        if abs(bounds0[0] - bounds0[1]) < EPS:
            return (-1, -1)
    
    #Then establish interval bounds 
    lowerB = [0, a0]
    upperB = [a0, 100]
    
    #Upper Bound
    a1 = a0
    step = STEP
    pos = POS.perf
    while abs(upperB[0] - upperB[1]) > EPS2:
        a1 = a1 + step
        accepted = pVals[pVals['Final_tstat'] > a1]
        
        #Determine position of alpha 
        if len(accepted.index) == 0:
            pos = POS.big
        else:
            pos = POS.perf
        
        #Determine what alpha to check next 
        if pos == POS.perf:
            upperB[0] = a1
            if a1 + abs(step * FACTOR2) >= upperB[1]:
                step = abs(a1 - upperB[1])/FACTOR2
            else:
                step = abs(step * FACTOR2) 
        elif pos == POS.big:
            upperB[1] = a1
            if (a1 - abs(step * FACTOR2)) <= upperB[0]:
                step = -1 * abs(a1 - upperB[0])/FACTOR2
            else:
                step = -1 * abs(step * FACTOR2) 
        else:
            assert False

    #Lower Bound
    a2 = a0
    if a2 - STEP > 1e-20:
        step = STEP
    else: 
        step = a2/FACTOR2 
    pos = POS.perf
    while abs(lowerB[0] - lowerB[1]) > EPS2:
        a2 = a2 - step
        accepted = pVals[pVals['Final_tstat'] > a2]
        
        #Determine position of alpha 
        accepted_sets = [str_2_pcp(a) for a in list(accepted.index)]
        causal_preds = set.intersection(*accepted_sets)
        if len(causal_preds) == 0:
            pos = POS.small 
        else:
            pos = POS.perf       
        
        #Determine what alpha to check next 
        if pos == POS.perf:
            lowerB[1] = a2
            if a2 - abs(step * FACTOR2) <= lowerB[0]:
                step = abs(a2 - lowerB[0])/FACTOR2
            else:
                step = abs(step * FACTOR2) 
        elif pos == POS.small:
            lowerB[0] = a2
            if (a1 + abs(step * FACTOR2)) >= lowerB[1]:
                step = -1 * abs(a2 - lowerB[1])/FACTOR2
            else:
                step = -1 * abs(step * FACTOR2) 
        else:
            assert False
    
    #Check if interval is too close to 0 to be meaningful 
    if flag:
        print(a2, a1)
    if a2 < amin: 
        return (-1, -1)
        
    #Establish 0-padding to interval
    interval = abs(a1 - a2)/5
    
    return (max(0, a2 - interval), a1 + interval)


# # File Generation

In [134]:
#Collect all files appropiate to each unique identifier 
rawres_files= []
for f in os.listdir(expdir):
    if ('rawres_' in f):
        rawres_files.append(f)

# Parameters

In [135]:
#Generate Alphas 
NUM_POINTS = 100
MIN_ALPHA = 1e-4

alphas = {}
for fname in rawres_files:
    f, d, s, e = get_hps_from_rawres(fname) 
    
    unid = '{}_{}_{}_{}'.format(f,d,s,e)
    try:
        pvals = json.load(open(os.path.join(expdir, fname), 'rb'))
        del pvals["()"]
    except:
        continue
    pvals = pd.DataFrame.from_dict(pvals, orient='index')
    alphas[(s, d, e)] = [x for x in alpha_tune(pvals, MIN_ALPHA)] + [NUM_POINTS]
    if (s == '587' and e =='native-country'):
        print(alpha_tune(pvals, MIN_ALPHA, flag=1))
    
alphas = pd.DataFrame(alphas).T
alphas.columns = ['start', 'stop', 'num_points']
alphas.index.names = ['seed', 'dataset', 'env']
alphas.head(1000)

0.12266386173665521 0.18143080376088624
(0.110910473331809, 0.19318419216573246)


start      stop  num_points
seed dataset env                                           
1000 adult   marital-status -1.000000 -1.000000       100.0
             native-country  1.355089  3.746657       100.0
             occupation     -1.000000 -1.000000       100.0
             relationship   -1.000000 -1.000000       100.0
             workclass       0.161204  2.858142       100.0
147  adult   marital-status -1.000000 -1.000000       100.0
             native-country  0.000000  0.034001       100.0
             occupation      0.000000  0.019522       100.0
             relationship   -1.000000 -1.000000       100.0
             workclass       0.420351  0.462722       100.0
256  adult   marital-status -1.000000 -1.000000       100.0
             native-country  0.107085  0.131384       100.0
             occupation     -1.000000 -1.000000       100.0
             relationship   -1.000000 -1.000000       100.0
             workclass       0.325479  1.760375       100.0
304  adult   marital-status -1.000000 -1.000000       100.0
             native-country  2.030473  2.859470       100.0
             occupation      0.000000  0.012866       100.0
             relationship   -1.000000 -1.000000       100.0
             workclass       0.496589  0.738079       100.0
52   adult   marital-status -1.000000 -1.000000       100.0
             native-country  0.464896  0.668941       100.0
             occupation     -1.000000 -1.000000       100.0
             relationship   -1.000000 -1.000000       100.0
             workclass       0.104356  0.219836       100.0
587  adult   marital-status -1.000000 -1.000000       100.0
             native-country  0.110910  0.193184       100.0
             occupation     -1.000000 -1.000000       100.0
             relationship   -1.000000 -1.000000       100.0
             workclass       0.185550  0.608483       100.0
737  adult   marital-status -1.000000 -1.000000       100.0
             native-country  0.115870  0.680514       100.0
             occupation     -1.000000 -1.000000       100.0
             relationship   -1.000000 -1.000000       100.0
             workclass       0.151461  0.233008       100.0
784  adult   marital-status -1.000000 -1.000000       100.0
             native-country  0.095347  0.189011       100.0
             occupation      0.000000  0.096277       100.0
             relationship   -1.000000 -1.000000       100.0
             workclass       0.000000  1.993170       100.0
8079 adult   marital-status -1.000000 -1.000000       100.0
             native-country  0.000000  2.685288       100.0
             occupation      0.006996  0.016291       100.0
             relationship   -1.000000 -1.000000       100.0
             workclass       0.896955  2.958019       100.0
990  adult   marital-status -1.000000 -1.000000       100.0
             native-country  0.319961  0.339691       100.0
             occupation     -1.000000 -1.000000       100.0
             relationship   -1.000000 -1.000000       100.0
             workclass       0.123644  0.134059       100.0

In [136]:
feateng = ['12']
dataset = ['adult']  #['adult', germanCredit']
seed = [str(x) for x in set(alphas.index.get_level_values(0))]  #, '8079', '52', '147', '256', '784', '990', '587', '304','737']
environment = [str(x) for x in set(alphas.index.get_level_values(2))] 

available_exps = itertools.product(feateng, dataset, seed, environment)

In [137]:
x_axis = {}  #x,y values  for plot of alpha vs #CPs 
y_axis = {}
CPid_results = {}  #Stores CPids of each expierment 
# assert len(list(itertools.product(feateng, dataset, seed, environment))) == 1

for exp in available_exps:
    for fname in rawres_files:
        f, d, s, e = get_hps_from_rawres(fname)  
        if (f == exp[0]) and (d == exp[1]) and (s == exp[2]) and (e == exp[3]):
            unid = '{}_{}_{}_{}'.format(f,d,s,e)
            try:
                pvals = json.load(open(os.path.join(expdir, fname), 'rb'))
                del pvals["()"]
            except:
                continue
            pvals = pd.DataFrame.from_dict(pvals, orient='index')
            
            #For Storing all the results 
            x_axis[(s, d, e)] = []
            y_axis[(s, d, e)] = []
            CPid_results[(s, d, e)] = Counter()
            norm = 0
            
            start, stop, num_points = alphas.loc[s, d, e][0], alphas.loc[s, d, e][1], alphas.loc[s, d, e][2]
            for a in np.linspace(start, stop, num_points): 
                accepted = pvals[pvals['Final_tstat'] > a]
                if len(accepted.index) > 100000:
                    raise ValueError('too many subsets: {}'.format(len(accepted.index)))
                
                accepted_sets = list(accepted.index)
                accepted_sets = [str_2_pcp(a) for a in accepted_sets]
                if len(accepted_sets) > 0:
                    pcps = set.intersection(*accepted_sets)
                else:
                    pcps = set([])
                
                #Number of Accepted Sets 
                x_axis[(s, d, e)].append(a)
                if len(accepted_sets) == 0:
                    y_axis[(s, d, e)].append(0)
                else:
                    y_axis[(s, d, e)].append(len(set.intersection(*accepted_sets)))
                    
                #Causal predictor  
                for pcp in pcps: 
                    CPid_results[(s, d, e)].update({pcp:1})



/Users/RobertAdragna/Documents/School/Fourth_Year/ESC499-Thesis/codebases/thesisenv/lib/python3.6/site-packages/ipykernel_launcher.py:25: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.


# Save Results

In [138]:
pickle.dump(x_axis, open(os.path.join(savedir, 'x_axis'), 'wb'))
pickle.dump(y_axis, open(os.path.join(savedir, 'y_axis'), 'wb'))
pickle.dump(CPid_results, open(os.path.join(savedir, 'CPid_results'), 'wb'))

In [141]:
print(y_axis[('1000', 'adult', 'native-country')])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


# Appendix

## CALIBRATION

In [139]:
# #Plot Accepted subsets vs Alpha for specified hyperparams 

# #fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(w*5, int(l/w)*5)) #Note - is +2 for reason


for exp in itertools.product(feateng, dataset, seed, environment):
    for fname in rawres_files:
        f, d, s, e = get_hps_from_rawres(fname)
        if (f == exp[0]) and (d == exp[1]) and (s == exp[2]) and (e == exp[3]):
            unid = '{}_{}_{}_{}'.format(f,d,s,e)
            try:
                pvals = json.load(open(os.path.join(expdir, fname), 'rb'))
                del pvals["()"]
            except:
                continue
            pvals = pd.DataFrame.from_dict(pvals, orient='index')
            
            start, stop, num_points = alphas.loc[s, d, e][0], alphas.loc[s, d, e][1], alphas.loc[s, d, e][2]
            for a in np.linspace(start, stop, num_points): 
                accepted = pvals[pvals['Final_tstat'] > a]
                if len(accepted.index) == 0:
                    print(a, unid, 0, 'null')
                elif len(accepted.index) < 1000:
                    accepted_sets = list(accepted.index)
                    accepted_sets = [str_2_pcp(a) for a in accepted_sets]
                    print(a, unid, len(accepted.index), set.intersection(*accepted_sets))
                else:
                    print(a, unid, len(accepted.index), 'too_many_intersections')
            
    
    print('#####################################')

/Users/RobertAdragna/Documents/School/Fourth_Year/ESC499-Thesis/codebases/thesisenv/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.


0.11587001891434183 12_adult_737_native-country 3 set()
0.12157349279387421 12_adult_737_native-country 3 set()
0.1272769666734066 12_adult_737_native-country 3 set()
0.13298044055293898 12_adult_737_native-country 3 set()
0.13868391443247136 12_adult_737_native-country 3 set()
0.14438738831200376 12_adult_737_native-country 3 set()
0.1500908621915361 12_adult_737_native-country 3 set()
0.15579433607106852 12_adult_737_native-country 2 set()
0.1614978099506009 12_adult_737_native-country 2 set()
0.16720128383013327 12_adult_737_native-country 2 set()
0.17290475770966568 12_adult_737_native-country 2 set()
0.17860823158919803 12_adult_737_native-country 2 set()
0.18431170546873044 12_adult_737_native-country 2 set()
0.1900151793482628 12_adult_737_native-country 2 set()
0.1957186532277952 12_adult_737_native-country 2 set()
0.2014221271073276 12_adult_737_native-country 1 {"'marital-status'", "'relationship'", "'education'"}
0.20712560098685995 12_adult_737_native-country 1 {"'marital-s

-1.0 12_adult_737_relationship 2047 too_many_intersections
-1.0 12_adult_737_relationship 2047 too_many_intersections
-1.0 12_adult_737_relationship 2047 too_many_intersections
-1.0 12_adult_737_relationship 2047 too_many_intersections
-1.0 12_adult_737_relationship 2047 too_many_intersections
-1.0 12_adult_737_relationship 2047 too_many_intersections
-1.0 12_adult_737_relationship 2047 too_many_intersections
-1.0 12_adult_737_relationship 2047 too_many_intersections
-1.0 12_adult_737_relationship 2047 too_many_intersections
-1.0 12_adult_737_relationship 2047 too_many_intersections
-1.0 12_adult_737_relationship 2047 too_many_intersections
-1.0 12_adult_737_relationship 2047 too_many_intersections
-1.0 12_adult_737_relationship 2047 too_many_intersections
-1.0 12_adult_737_relationship 2047 too_many_intersections
-1.0 12_adult_737_relationship 2047 too_many_intersections
-1.0 12_adult_737_relationship 2047 too_many_intersections
-1.0 12_adult_737_relationship 2047 too_many_intersectio

-1.0 12_adult_737_occupation 2047 too_many_intersections
-1.0 12_adult_737_occupation 2047 too_many_intersections
-1.0 12_adult_737_occupation 2047 too_many_intersections
-1.0 12_adult_737_occupation 2047 too_many_intersections
-1.0 12_adult_737_occupation 2047 too_many_intersections
-1.0 12_adult_737_occupation 2047 too_many_intersections
-1.0 12_adult_737_occupation 2047 too_many_intersections
-1.0 12_adult_737_occupation 2047 too_many_intersections
-1.0 12_adult_737_occupation 2047 too_many_intersections
-1.0 12_adult_737_occupation 2047 too_many_intersections
-1.0 12_adult_737_occupation 2047 too_many_intersections
-1.0 12_adult_737_occupation 2047 too_many_intersections
-1.0 12_adult_737_occupation 2047 too_many_intersections
-1.0 12_adult_737_occupation 2047 too_many_intersections
-1.0 12_adult_737_occupation 2047 too_many_intersections
-1.0 12_adult_737_occupation 2047 too_many_intersections
-1.0 12_adult_737_occupation 2047 too_many_intersections
-1.0 12_adult_737_occupation 20

0.1842805343344055 12_adult_784_native-country 0 null
0.18522663373643097 12_adult_784_native-country 0 null
0.18617273313845648 12_adult_784_native-country 0 null
0.18711883254048195 12_adult_784_native-country 0 null
0.18806493194250745 12_adult_784_native-country 0 null
0.18901103134453295 12_adult_784_native-country 0 null
#####################################
-1.0 12_adult_784_relationship 2047 too_many_intersections
-1.0 12_adult_784_relationship 2047 too_many_intersections
-1.0 12_adult_784_relationship 2047 too_many_intersections
-1.0 12_adult_784_relationship 2047 too_many_intersections
-1.0 12_adult_784_relationship 2047 too_many_intersections
-1.0 12_adult_784_relationship 2047 too_many_intersections
-1.0 12_adult_784_relationship 2047 too_many_intersections
-1.0 12_adult_784_relationship 2047 too_many_intersections
-1.0 12_adult_784_relationship 2047 too_many_intersections
-1.0 12_adult_784_relationship 2047 too_many_intersections
-1.0 12_adult_784_relationship 2047 too_man

...

0.0 12_adult_147_native-country 2047 too_many_intersections
0.0003434429045089267 12_adult_147_native-country 1088 too_many_intersections
0.0006868858090178534 12_adult_147_native-country 927 set()
0.00103032871352678 12_adult_147_native-country 847 set()
0.0013737716180357068 12_adult_147_native-country 782 set()
0.0017172145225446335 12_adult_147_native-country 739 set()
0.00206065742705356 12_adult_147_native-country 705 set()
0.002404100331562487 12_adult_147_native-country 672 set()
0.0027475432360714136 12_adult_147_native-country 633 set()
0.0030909861405803403 12_adult_147_native-country 612 set()
0.003434429045089267 12_adult_147_native-country 587 set()
0.003777871949598194 12_adult_147_native-country 552 set()
0.00412131485410712 12_adult_147_native-country 529 set()
0.004464757758616047 12_adult_147_native-country 507 set()
0.004808200663124974 12_adult_147_native-country 494 set()
0.005151643567633901 12_adult_147_native-country 474 set()
0.005495086472142827 12_adult_147_

0.4224912153598335 12_adult_147_workclass 2 set()
0.4229192022730907 12_adult_147_workclass 2 set()
0.4233471891863479 12_adult_147_workclass 2 set()
0.4237751760996051 12_adult_147_workclass 2 set()
0.4242031630128622 12_adult_147_workclass 2 set()
0.4246311499261194 12_adult_147_workclass 2 set()
0.4250591368393766 12_adult_147_workclass 2 set()
0.42548712375263376 12_adult_147_workclass 2 set()
0.42591511066589094 12_adult_147_workclass 2 set()
0.42634309757914807 12_adult_147_workclass 2 set()
0.42677108449240525 12_adult_147_workclass 2 set()
0.42719907140566243 12_adult_147_workclass 2 set()
0.4276270583189196 12_adult_147_workclass 2 set()
0.4280550452321768 12_adult_147_workclass 2 set()
0.428483032145434 12_adult_147_workclass 2 set()
0.4289110190586911 12_adult_147_workclass 2 set()
0.4293390059719483 12_adult_147_workclass 2 set()
0.4297669928852055 12_adult_147_workclass 2 set()
0.43019497979846266 12_adult_147_workclass 2 set()
0.43062296671171985 12_adult_147_workclass 2 

-1.0 12_adult_147_marital-status 2047 too_many_intersections
-1.0 12_adult_147_marital-status 2047 too_many_intersections
-1.0 12_adult_147_marital-status 2047 too_many_intersections
-1.0 12_adult_147_marital-status 2047 too_many_intersections
-1.0 12_adult_147_marital-status 2047 too_many_intersections
-1.0 12_adult_147_marital-status 2047 too_many_intersections
-1.0 12_adult_147_marital-status 2047 too_many_intersections
-1.0 12_adult_147_marital-status 2047 too_many_intersections
-1.0 12_adult_147_marital-status 2047 too_many_intersections
-1.0 12_adult_147_marital-status 2047 too_many_intersections
-1.0 12_adult_147_marital-status 2047 too_many_intersections
-1.0 12_adult_147_marital-status 2047 too_many_intersections
-1.0 12_adult_147_marital-status 2047 too_many_intersections
-1.0 12_adult_147_marital-status 2047 too_many_intersections
-1.0 12_adult_147_marital-status 2047 too_many_intersections
-1.0 12_adult_147_marital-status 2047 too_many_intersections
-1.0 12_adult_147_marita

-1.0 12_adult_52_relationship 2047 too_many_intersections
-1.0 12_adult_52_relationship 2047 too_many_intersections
-1.0 12_adult_52_relationship 2047 too_many_intersections
-1.0 12_adult_52_relationship 2047 too_many_intersections
-1.0 12_adult_52_relationship 2047 too_many_intersections
-1.0 12_adult_52_relationship 2047 too_many_intersections
-1.0 12_adult_52_relationship 2047 too_many_intersections
-1.0 12_adult_52_relationship 2047 too_many_intersections
-1.0 12_adult_52_relationship 2047 too_many_intersections
-1.0 12_adult_52_relationship 2047 too_many_intersections
-1.0 12_adult_52_relationship 2047 too_many_intersections
-1.0 12_adult_52_relationship 2047 too_many_intersections
-1.0 12_adult_52_relationship 2047 too_many_intersections
-1.0 12_adult_52_relationship 2047 too_many_intersections
-1.0 12_adult_52_relationship 2047 too_many_intersections
-1.0 12_adult_52_relationship 2047 too_many_intersections
-1.0 12_adult_52_relationship 2047 too_many_intersections
-1.0 12_adult_

0.18017595660791855 12_adult_52_workclass 1 {"'marital-status'"}
0.18134241650005184 12_adult_52_workclass 1 {"'marital-status'"}
0.1825088763921851 12_adult_52_workclass 1 {"'marital-status'"}
0.18367533628431837 12_adult_52_workclass 1 {"'marital-status'"}
0.18484179617645163 12_adult_52_workclass 1 {"'marital-status'"}
0.1860082560685849 12_adult_52_workclass 1 {"'marital-status'"}
0.18717471596071816 12_adult_52_workclass 1 {"'marital-status'"}
0.18834117585285143 12_adult_52_workclass 1 {"'marital-status'"}
0.18950763574498472 12_adult_52_workclass 1 {"'marital-status'"}
0.19067409563711799 12_adult_52_workclass 1 {"'marital-status'"}
0.19184055552925125 12_adult_52_workclass 1 {"'marital-status'"}
0.19300701542138454 12_adult_52_workclass 1 {"'marital-status'"}
0.1941734753135178 12_adult_52_workclass 1 {"'marital-status'"}
0.19533993520565107 12_adult_52_workclass 1 {"'marital-status'"}
0.19650639509778434 12_adult_52_workclass 1 {"'marital-status'"}
0.1976728549899176 12_adult_